In [3]:
#Расчёт пластинчатого ТО
import matplotlib.pyplot as pt
import numpy as n
from CoolProp.CoolProp import PropsSI as p
from scipy.optimize import root
import math as m

#Исходные данные:
G1 = 636.690003
T11 = 663.629872
T12 = 77

G2 = 543.18
T21 = 60.2275183


P11=3000000
P21=30000000
P12 = P11
P22 = P21

##############################################
CO2  = 'REFPROP::CO2'
H2O  = 'REFPROP::WATER'
O2   = 'REFPROP::Oxygen'
MIX1 =  'REFPROP::CO2[0.931]&WATER[0.069]'
MIX2 =  'REFPROP::CO2[0.993]&WATER[0.007]'
MIX05 =  'REFPROP::CO2[0.95]&WATER[0.05]' #Для транспортных свойств


H11  = p('H','T', T11+273.15, 'P', P11, MIX1)
H12  = p('H','T', T12+273.15, 'P', P11, MIX1)
H21  = p('H','T', T21+273.15, 'P', P21, MIX2)

#Разбиение по энтальпии:
s    = 100 #Кол-во шагов
step = (H11-H12)/(s) 

#Нулевые массивы:
t1   = n.zeros(s+1)
t2   = n.zeros(s+1)
Q    = n.zeros(s+1)

#Противоток:
h11 = H11
i=0
for i in range(s+1):
    t1[i] = p('T','H', h11, 'P', P11, MIX1)-273.15 
    h12 = h11 - step
    dQ = G1 * (h11-h12)
    if i < s:
        Q[i+1]=Q[i]+dQ
    h11 = h12
h21 = H21
i=0
for i in range(s+1):
    t2[s-i] = p('T','H', h21, 'P', P21, MIX2)-273.15  
    h22 = h21+((Q[s-i]-Q[s-i-1])/G2)
    h21 = h22
DT=t1-t2
##############################################
print(min(DT))

t1av = n.zeros(s)
t2av = n.zeros(s)
ro1 = n.zeros(s)
ro2 = n.zeros(s)
nu1 = n.zeros(s)
nu2 = n.zeros(s)
lambda1 = n.zeros(s)
lambda2 = n.zeros(s)
Pr1 = n.zeros(s)
Pr2 = n.zeros(s)
DTav = n.zeros(s)
w1 = n.zeros(s)
w2= n.zeros(s)
Re1= n.zeros(s)
Re2= n.zeros(s)
Nu1= n.zeros(s)
Nu2= n.zeros(s)
f1= n.zeros(s)
f2= n.zeros(s)
fNu1= n.zeros(s)
fNu2= n.zeros(s)
alpha1= n.zeros(s)
alpha2= n.zeros(s)
K= n.zeros(s)
F= n.zeros(s+1)
L= n.zeros(s+1)
P1= n.zeros(s+1)
P2= n.zeros(s+1)


ddP1     =n.zeros(10000)
ddP2     =n.zeros(10000)
ddP4     =n.zeros(10000)
ddP6     =n.zeros(10000)
ddP8     =n.zeros(10000)
ddP10    =n.zeros(10000)
Vv     =n.zeros(10000)
DdP1   =n.zeros(10000)
DdP2   =n.zeros(10000)
Ff     =n.zeros(10000)
Ll     =n.zeros(10000)
Nnsumm =n.zeros(10000)

#Для каждого участка:
for i in range(s):
    #Свойства
    t1av[i]   =(t1[i]+t1[i+1])/2
    t2av[i]   =(t2[i]+t2[i+1])/2
    ro1[i]    =p('D','T', t1av[i]+273.15, 'P', P11, MIX1)
    ro2[i]    =p('D','T', t2av[i]+273.15, 'P', P21, MIX2)
    nu1[i]    =p('V','T', t1av[i]+273.15, 'P', P11, MIX05)/ro1[i]
    nu2[i]    =p('V','T', t2av[i]+273.15, 'P', P21, MIX05)/ro2[i]
    lambda1[i]=p('L','T', t1av[i]+273.15, 'P', P11, MIX05)
    lambda2[i]=p('L','T', t2av[i]+273.15, 'P', P21, MIX05)
    Pr1[i]    =p('PRANDTL','T', t1av[i]+273.15, 'P', P11, MIX05)
    Pr2[i]    =p('PRANDTL','T', t2av[i]+273.15, 'P', P21, MIX05)
    DTav[i]   =t1av[i]-t2av[i]
#От выпадающих Pr:
for i in range(1,s):
    if abs(Pr1[i]-Pr1[i-1])>Pr1[i-1]:
        Pr1[i]=p('PRANDTL','T', t1[i]+273.15, 'P', P11, CO2)
    if abs(Pr2[i]-Pr2[i-1])>Pr2[i-1]:
        Pr2[i]=p('PRANDTL','T', t2[i]+273.15, 'P', P21, CO2)

#Геометрия
lambdaw = 20   #теплопроводность металла
dc = 0.002     #диаметр канала
t = 0.0015     #толщина пластины
wpl = 1        #ширина пластины
pc = 0.0024    #шаг канала
delta = 10**-5 #Шероховатость
dh = 4 *n.pi *(dc**2)/(8*((n.pi*dc/2)+dc))
fproh = 416*(n.pi*(dc**2)/8)
fl = 416*((n.pi*dc/2)+dc)

#Геометрия "выхода":
roout1 = p('D','T', t1[s]+273.15, 'P', P11, MIX1)
roout2 = p('D','T', t2[s]+273.15, 'P', P21, MIX2)


print('+')
j=0



wout1 = 2
Fproh1 = G1/(roout1*wout1)
Npl1 = m.ceil(Fproh1/fproh)
Npl2 = m.ceil(Npl1/2)
Nsumm = Npl1+Npl2
Fproh2 = Npl2*fproh
Fl = ((Npl1+Npl2)/2)*fl


P1[0]=P11
P2[0]=P21
#Для каждого участка:
for i in range(s):
    w1[i] = G1/(ro1[i]*Fproh1)
    w2[i] = G2/(ro2[i]*Fproh2)
    Re1[i] = w1[i]*dh/nu1[i]
    Re2[i] = w2[i]*dh/nu2[i]
    
    
    if Re1[i] >= 2300 :
        fNu1[i] = (1/(1.8*n.log10(Re1[i])-1.5))**2
        Nu1[i] = ((fNu1[i]/8)*(Re1[i]-1000)*Pr1[i])/(1+ 12.7*(Pr1[i]**(2/3)-1)*((fNu1[i]/8)**(1/2)))
        def func(x):
            left = x
            right = 1/(2*n.log10((2.51/(Re1[i]*((x)**(1/2))))+(delta/3.7)))**2
            return left-right
        sol = root(func, 0.1)
        f1[i]=float(sol.x)
    if Re1[i] < 2300:
        Nu1[i] = 4.089
        f1[i] = 64/Re1[i]
    if Re2[i] >= 2300:
        fNu2[i] = (1/(1.8*n.log10(Re2[i])-1.5))**2
        Nu2[i] = ((fNu2[i]/8)*(Re2[i]-1000)*Pr2[i])/(1+ 12.7*(Pr2[i]**(2/3)-1)*((fNu2[i]/8)**(1/2)))
        def func(x):
            left = x
            right = 1/(2*n.log10((2.51/(Re2[i]*((x)**(1/2))))+(delta/3.7)))**2
            return left-right
        sol = root(func, 0.1)
        f2[i]=float(sol.x)
    if Re2[i] < 2300:
        Nu2[i] = 4.089
        f2[i] = 64/Re2[i]
    alpha1[i] = Nu1[i]*lambda1[i]/dh
    alpha2[i] = Nu2[i]*lambda2[i]/dh
    K[i] = 1/((1/alpha1[i])+(t/lambdaw)+(1/alpha2[i]))
    dF = (Q[i+1]-Q[i])/(K[i]*DTav[i])
    dL = dF/Fl
    if i < s:
        F[i+1]=F[i]+dF
    if i < s:
        L[i+1]=L[i]+dL
    dP1 = f1[i]*dL/dh*ro1[i]*w1[i]**2/2
    dP2 = f2[i]*dL/dh*ro2[i]*w2[i]**2/2
    if i < s:
        P1[i+1]=P1[i]-dP1
        P2[i+1]=P2[i]-dP2
    DP1 = P1[0]-P1[s]
    DP2 = P2[0]-P2[s]
print("++")

4.774817591503393
+
++
